# Import

In [ ]:
import numpy as np
import pandas as pd

# Mount drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Preprocessing

In [ ]:
# df_unsupervised = pd.read_csv('/content/drive/MyDrive/PoliceProject/checkpoint-4/officer_data_allegations.csv')
# df_unsupervised

In [ ]:
df_supervised = pd.read_csv('/content/drive/MyDrive/PoliceProject/checkpoint-4/police_allegations_with_officer_info.csv.csv')
print(df_supervised.columns)
df_supervised.head()

Index(['id', 'start_date', 'end_date', 'officer_age', 'recc_finding',
       'recc_outcome', 'final_finding', 'final_outcome', 'final_outcome_class',
       'allegation_category_id', 'officer_id', 'disciplined', 'created_at',
       'updated_at', 'allegation_id', 'id.1', 'gender', 'race',
       'appointed_date', 'rank', 'active', 'birth_year', 'first_name',
       'last_name', 'tags', 'middle_initial', 'suffix_name',
       'resignation_date', 'complaint_percentile', 'middle_initial2',
       'civilian_allegation_percentile', 'honorable_mention_percentile',
       'internal_allegation_percentile', 'trr_percentile', 'allegation_count',
       'sustained_count', 'civilian_compliment_count', 'current_badge',
       'current_salary', 'discipline_count', 'honorable_mention_count',
       'last_unit_id', 'major_award_count', 'trr_count', 'unsustained_count',
       'has_unique_name', 'created_at.1', 'updated_at.1', 'incident_date'],
      dtype='object')


,id,start_date,end_date,officer_age,recc_finding,recc_outcome,final_finding,final_outcome,final_outcome_class,allegation_category_id,...,discipline_count,honorable_mention_count,last_unit_id,major_award_count,trr_count,unsustained_count,has_unique_name,created_at.1,updated_at.1,incident_date
0,219949,2011-09-14,2012-12-06,NaN,SU,Reprimand,SU,Penalty Not Served,NaN,199.0,...,4,5,3.0,0,0,1,True,2019-01-09 04:41:58.08732+00,2021-01-04 00:58:43.225358+00,2011-04-13 00:00:00+00
1,221159,2011-11-28,2011-12-23,NaN,NaN,Unknown,NaN,Unknown,NaN,194.0,...,1,30,153.0,1,2,7,True,2019-01-09 04:41:58.08732+00,2021-01-04 00:58:42.591928+00,2011-11-15 00:00:00+00
2,222506,2012-01-20,2014-06-02,NaN,SU,Reprimand,SU,Reprimand,NaN,91.0,...,2,25,179.0,0,2,2,True,2019-01-09 04:41:58.08732+00,2021-01-04 00:58:43.225358+00,2012-01-20 00:00:00+00
3,223815,2012-04-26,2015-02-03,NaN,NaN,Unknown,NaN,Unknown,NaN,NaN,...,3,27,11.0,1,4,1,True,2019-01-09 04:41:58.08732+00,2021-01-04 00:58:42.591928+00,2012-04-25 00:00:00+00
4,187241,2007-08-17,2008-09-05,NaN,SU,1 Day Suspension,SU,Reprimand,NaN,173.0,...,3,3,20.0,0,11,1,True,2019-01-09 04:41:58.08732+00,2019-01-09 04:41:58.54702+00,2007-08-09 00:00:00+00


In [ ]:
df_supervised_2 = pd.read_csv('/content/drive/MyDrive/PoliceProject/checkpoint-4/civlian_allegaitons_with_officer_info.csv')
df_supervised_2.columns

Index(['id', 'start_date', 'end_date', 'officer_age', 'recc_finding',
       'recc_outcome', 'final_finding', 'final_outcome', 'final_outcome_class',
       'allegation_category_id', 'officer_id', 'disciplined', 'created_at',
       'updated_at', 'allegation_id', 'id.1', 'gender', 'race',
       'appointed_date', 'rank', 'active', 'birth_year', 'first_name',
       'last_name', 'tags', 'middle_initial', 'suffix_name',
       'resignation_date', 'complaint_percentile', 'middle_initial2',
       'civilian_allegation_percentile', 'honorable_mention_percentile',
       'internal_allegation_percentile', 'trr_percentile', 'allegation_count',
       'sustained_count', 'civilian_compliment_count', 'current_badge',
       'current_salary', 'discipline_count', 'honorable_mention_count',
       'last_unit_id', 'major_award_count', 'trr_count', 'unsustained_count',
       'has_unique_name', 'created_at.1', 'updated_at.1', 'incident_date'],
      dtype='object')

In [ ]:
# make a dict mapping from officer id to a list of civilian complaint dates
officer_to_incidents_dict = {}
for ind in df_supervised_2.index:
    curr_id = df_supervised_2['officer_id'][ind] 
    curr_date = pd.to_datetime( df_supervised_2['incident_date'][ind] )
    if curr_id in officer_to_incidents_dict.keys():
      officer_to_incidents_dict[curr_id].append(curr_date)
    else:
      officer_to_incidents_dict[curr_id] = [curr_date]

# test code
(officer_to_incidents_dict[2337][0] - officer_to_incidents_dict[2337][1]).days

-2085

In [ ]:
officer_to_incidents_dict

{3012: [Timestamp('2000-06-16 00:00:00+0000', tz='UTC')],
 17834: [Timestamp('2007-08-09 00:00:00+0000', tz='UTC'),
  Timestamp('2008-01-20 00:00:00+0000', tz='UTC'),
  Timestamp('2010-07-01 00:00:00+0000', tz='UTC'),
  Timestamp('2010-07-19 00:00:00+0000', tz='UTC'),
  Timestamp('2010-10-24 00:00:00+0000', tz='UTC'),
  Timestamp('2012-07-23 00:00:00+0000', tz='UTC')],
 3869: [Timestamp('2007-10-03 00:00:00+0000', tz='UTC')],
 3753: [Timestamp('2007-02-21 00:00:00+0000', tz='UTC'),
  Timestamp('2008-06-04 00:00:00+0000', tz='UTC'),
  Timestamp('2009-03-17 00:00:00+0000', tz='UTC'),
  Timestamp('2005-12-25 00:00:00+0000', tz='UTC')],
 8153: [Timestamp('2007-11-13 00:00:00+0000', tz='UTC'),
  Timestamp('2009-06-08 00:00:00+0000', tz='UTC'),
  Timestamp('2002-03-13 00:00:00+0000', tz='UTC'),
  Timestamp('2002-12-23 00:00:00+0000', tz='UTC')],
 32208: [Timestamp('2008-02-09 00:00:00+0000', tz='UTC'),
  Timestamp('2002-07-31 00:00:00+0000', tz='UTC')],
 29856: [Timestamp('2011-04-13 00:00:0

In [ ]:
# add a new column to df_supervised (1 or 0) for whether that officer has a civ complaint in 2 years
# loop over df, get current off id, lookup in dict, check if there's a date within 2 years of that police cmplt
# on each row, need officer id ('officer_id') and date of incident ('incident_date')

def extract_incident_bool(row):
  officer_id = row['officer_id']
  off_incident_date = pd.to_datetime( row['incident_date'] )
  civ_incidents_list = officer_to_incidents_dict[officer_id] # list of timestamps

  # loop over civ_incidents_list, check if i - off_incident_date < 2 years (aka 730days)?
  for i in civ_incidents_list:
    days_elapsed = (i - off_incident_date).days
    if days_elapsed > 0 and days_elapsed < (365)*2: 
      return 1
  return 0

df_supervised['incident_in_2_years_bool'] = df_supervised.apply(extract_incident_bool, axis=1)
df_supervised

,id,start_date,end_date,officer_age,recc_finding,recc_outcome,final_finding,final_outcome,final_outcome_class,allegation_category_id,...,honorable_mention_count,last_unit_id,major_award_count,trr_count,unsustained_count,has_unique_name,created_at.1,updated_at.1,incident_date,incident_in_2_years_bool
0,219949,2011-09-14,2012-12-06,NaN,SU,Reprimand,SU,Penalty Not Served,NaN,199.0,...,5,3.0,0,0,1,True,2019-01-09 04:41:58.08732+00,2021-01-04 00:58:43.225358+00,2011-04-13 00:00:00+00,0
1,221159,2011-11-28,2011-12-23,NaN,NaN,Unknown,NaN,Unknown,NaN,194.0,...,30,153.0,1,2,7,True,2019-01-09 04:41:58.08732+00,2021-01-04 00:58:42.591928+00,2011-11-15 00:00:00+00,0
2,222506,2012-01-20,2014-06-02,NaN,SU,Reprimand,SU,Reprimand,NaN,91.0,...,25,179.0,0,2,2,True,2019-01-09 04:41:58.08732+00,2021-01-04 00:58:43.225358+00,2012-01-20 00:00:00+00,0
3,223815,2012-04-26,2015-02-03,NaN,NaN,Unknown,NaN,Unknown,NaN,NaN,...,27,11.0,1,4,1,True,2019-01-09 04:41:58.08732+00,2021-01-04 00:58:42.591928+00,2012-04-25 00:00:00+00,0
4,187241,2007-08-17,2008-09-05,NaN,SU,1 Day Suspension,SU,Reprimand,NaN,173.0,...,3,20.0,0,11,1,True,2019-01-09 04:41:58.08732+00,2019-01-09 04:41:58.54702+00,2007-08-09 00:00:00+00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14520,234260,2014-06-06,NaN,NaN,NaN,Unknown,NaN,Unknown,NaN,NaN,...,2,17.0,0,0,6,True,2019-01-09 04:41:58.08732+00,2021-01-04 00:58:42.591928+00,2014-06-05 00:00:00+00,1
14521,234548,2014-06-27,NaN,NaN,NaN,Unknown,NaN,Unknown,NaN,NaN,...,41,10.0,0,2,11,False,2019-01-09 04:41:58.08732+00,2021-01-04 00:58:42.591928+00,2013-08-18 00:00:00+00,0
14522,236806,2014-12-24,NaN,NaN,NaN,Unknown,NaN,Unknown,NaN,NaN,...,49,7.0,0,17,6,True,2019-01-09 04:41:58.08732+00,2021-01-04 00:58:42.591928+00,2014-11-03 00:00:00+00,0
14523,117922,2000-01-04,2001-09-26,NaN,SU,2 Day Suspension,SU,Reprimand,NaN,189.0,...,13,3.0,0,0,3,True,2019-01-09 04:41:58.08732+00,2021-01-04 00:58:43.151646+00,2000-01-04 00:00:00+00,1


In [ ]:
df_supervised.to_csv('/content/drive/MyDrive/PoliceProject/checkpoint-4/unclean_supervised_data.csv')


In [ ]:
half_clean_df = pd.read_csv('/content/drive/MyDrive/PoliceProject/checkpoint-4/half-clean-supervised-data.csv')
half_clean_df.head()

,Column,id,recc_finding,recc_outcome,final_finding,final_outcome,allegation_category_id,officer_id,disciplined,allegation_id,...,sustained_count,civilian_compliment_count,current_salary,discipline_count,honorable_mention_count,major_award_count,trr_count,unsustained_count,incident_date,incident_in_2_years_bool
0,0,219949,SU,Reprimand,SU,Penalty Not Served,199.0,29856,False,1048510,...,5,0,87006.0,4,5,0,0,1,2011-04-13 00:00:00+00,0
1,1,221159,NaN,Unknown,NaN,Unknown,194.0,31923,NaN,1050325,...,1,8,107988.0,1,30,1,2,7,2011-11-15 00:00:00+00,0
2,2,222506,SU,Reprimand,SU,Reprimand,91.0,25965,True,1051363,...,2,1,107988.0,2,25,0,2,2,2012-01-20 00:00:00+00,0
3,3,223815,NaN,Unknown,NaN,Unknown,NaN,4278,NaN,1053605,...,3,2,90024.0,3,27,1,4,1,2012-04-25 00:00:00+00,0
4,4,187241,SU,1 Week or less Suspension,SU,Reprimand,173.0,17834,True,1008522,...,4,2,78012.0,3,3,0,11,1,2007-08-09 00:00:00+00,1


In [ ]:
half_clean_df.columns

Index(['Column', 'id', 'recc_finding', 'recc_outcome', 'final_finding',
       'final_outcome', 'allegation_category_id', 'officer_id', 'disciplined',
       'allegation_id', 'gender', 'race', 'appointed_date', 'rank',
       'birth_year', 'complaint_percentile', 'civilian_allegation_percentile',
       'honorable_mention_percentile', 'internal_allegation_percentile',
       'trr_percentile', 'allegation_count', 'sustained_count',
       'civilian_compliment_count', 'current_salary', 'discipline_count',
       'honorable_mention_count', 'major_award_count', 'trr_count',
       'unsustained_count', 'incident_date', 'incident_in_2_years_bool'],
      dtype='object')

In [ ]:
half_clean_df['appointed_date']

0        1999-03-08
1        1995-12-04
2        1995-09-05
3        2001-02-05
4        2001-05-29
            ...    
14210    1997-05-05
14211    1998-10-26
14212    2001-07-30
14213    1998-08-31
14214    1986-06-16
Name: appointed_date, Length: 14215, dtype: object

In [ ]:
# birth year -> age (subtract birth year from incident year)
def extract_age(row):
  birth_year = row['birth_year']
  incident_year = pd.to_datetime( row['incident_date'] ).year
  return incident_year - birth_year

half_clean_df['age_at_time_of_incident'] = half_clean_df.apply(extract_age, axis=1)

In [ ]:
# appointed_date -> days experience
def extract_days_experience(row):
  appointed_date = pd.to_datetime( row['appointed_date'] )
  incident_date = pd.to_datetime( pd.to_datetime( row['incident_date'] ).date() )
  return (incident_date - appointed_date).days

half_clean_df['days_experience'] = half_clean_df.apply(extract_days_experience, axis=1)

In [ ]:
half_clean_df

,Column,id,recc_finding,recc_outcome,final_finding,final_outcome,allegation_category_id,officer_id,disciplined,allegation_id,...,current_salary,discipline_count,honorable_mention_count,major_award_count,trr_count,unsustained_count,incident_date,incident_in_2_years_bool,age_at_time_of_incident,days_experience
0,0,219949,SU,Reprimand,SU,Penalty Not Served,199.0,29856,False,1048510,...,87006.0,4,5,0,0,1,2011-04-13 00:00:00+00,0,49.0,4419.0
1,1,221159,NaN,Unknown,NaN,Unknown,194.0,31923,NaN,1050325,...,107988.0,1,30,1,2,7,2011-11-15 00:00:00+00,0,43.0,5825.0
2,2,222506,SU,Reprimand,SU,Reprimand,91.0,25965,True,1051363,...,107988.0,2,25,0,2,2,2012-01-20 00:00:00+00,0,41.0,5981.0
3,3,223815,NaN,Unknown,NaN,Unknown,NaN,4278,NaN,1053605,...,90024.0,3,27,1,4,1,2012-04-25 00:00:00+00,0,35.0,4097.0
4,4,187241,SU,1 Week or less Suspension,SU,Reprimand,173.0,17834,True,1008522,...,78012.0,3,3,0,11,1,2007-08-09 00:00:00+00,1,29.0,2263.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14210,14520,234260,NaN,Unknown,NaN,Unknown,NaN,5405,NaN,1069623,...,93354.0,8,2,0,0,6,2014-06-05 00:00:00+00,1,44.0,6240.0
14211,14521,234548,NaN,Unknown,NaN,Unknown,NaN,7201,NaN,1070031,...,104628.0,0,41,0,2,11,2013-08-18 00:00:00+00,0,38.0,5410.0
14212,14522,236806,NaN,Unknown,NaN,Unknown,NaN,29380,NaN,1073092,...,104628.0,1,49,0,17,6,2014-11-03 00:00:00+00,0,47.0,4844.0
14213,14523,117922,SU,1 Week or less Suspension,SU,Reprimand,189.0,24772,True,259088,...,89130.0,2,13,0,0,3,2000-01-04 00:00:00+00,1,47.0,491.0


In [ ]:
# load db table, make dict, loop over df and make new category column?
cats = pd.read_csv('/content/drive/MyDrive/PoliceProject/checkpoint-4/data_allegationcategory.csv')
cats.head()

,id,category_code,category,allegation_name,on_duty,citizen_dept,created_at,updated_at
0,1,001,Operation/Personnel Violations,Unfit For Duty,True,dept,2019-01-09 04:41:36.478496 +00:00,2019-01-09 04:41:36.629666 +00:00
1,30,019B,Operation/Personnel Violations,Reports - Failure To Report Incident,True,dept,2019-01-09 04:41:36.478496 +00:00,2019-01-09 04:41:36.629666 +00:00
2,31,01A,Verbal Abuse,Use Of Profanity,True,citizen,2019-01-09 04:41:36.478496 +00:00,2019-01-09 04:41:36.629666 +00:00
3,32,01B,Verbal Abuse,"Racial / Ethnic, Etc.",True,citizen,2019-01-09 04:41:36.478496 +00:00,2019-01-09 04:41:36.629666 +00:00
4,33,01C,Verbal Abuse,Sexual Orientation,True,citizen,2019-01-09 04:41:36.478496 +00:00,2019-01-09 04:41:36.629666 +00:00


In [ ]:
category_dict = {}
# map category 'id' to 'category'
for idx in cats.index:
  id = cats['id'][idx]
  category_dict[id] = cats['category'][idx]

category_dict.keys()

dict_keys([1, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 416, 110, 111, 284, 83, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 2

In [ ]:
# allegation_category_id -> category_name  (use table from db)
def extract_category_name(row):
  try:
    cat_name = category_dict[row['allegation_category_id']]
  except: # catch nan
    cat_name = "Unknown"
  return cat_name

half_clean_df['category_name'] = half_clean_df.apply(extract_category_name, axis=1)
half_clean_df.head()

,Column,id,recc_finding,recc_outcome,final_finding,final_outcome,allegation_category_id,officer_id,disciplined,allegation_id,...,discipline_count,honorable_mention_count,major_award_count,trr_count,unsustained_count,incident_date,incident_in_2_years_bool,age_at_time_of_incident,days_experience,category_name
0,0,219949,SU,Reprimand,SU,Penalty Not Served,199.0,29856,False,1048510,...,4,5,0,0,1,2011-04-13 00:00:00+00,0,49.0,4419.0,Operation/Personnel Violations
1,1,221159,NaN,Unknown,NaN,Unknown,194.0,31923,NaN,1050325,...,1,30,1,2,7,2011-11-15 00:00:00+00,0,43.0,5825.0,Operation/Personnel Violations
2,2,222506,SU,Reprimand,SU,Reprimand,91.0,25965,True,1051363,...,2,25,0,2,2,2012-01-20 00:00:00+00,0,41.0,5981.0,Lockup Procedures
3,3,223815,NaN,Unknown,NaN,Unknown,NaN,4278,NaN,1053605,...,3,27,1,4,1,2012-04-25 00:00:00+00,0,35.0,4097.0,Unknown
4,4,187241,SU,1 Week or less Suspension,SU,Reprimand,173.0,17834,True,1008522,...,3,3,0,11,1,2007-08-09 00:00:00+00,1,29.0,2263.0,Conduct Unbecoming (Off-Duty)


In [ ]:
# TODO: Impute Salary blanks with mean of column (https://www.geeksforgeeks.org/how-to-fill-nan-values-with-mean-in-pandas/)
print("NANs in column: ", half_clean_df['current_salary'].isna().sum())

salary_mean = half_clean_df['current_salary'].mean()
half_clean_df['current_salary'].fillna(value=salary_mean, inplace=True)

print("NANs in column: ", half_clean_df['current_salary'].isna().sum())

NANs in column:  508
NANs in column:  0


# Done

In [ ]:
half_clean_df.to_csv('/content/drive/MyDrive/PoliceProject/checkpoint-4/clean_supervised_data.csv')